In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')
from jaqs_fxdayu.data.dataservice import LocalDataService
ds = LocalDataService()
from time import time
## 加freq参数


In [2]:
#通过测试报告，看多空信号的绩效，这里是定义报告格式
from jaqs_fxdayu.research import TimingDigger

#td = TimingDigger(output_folder=".", output_format='pdf',signal_name='alpha60')

def TimingSignal(td, dv, long='Long', short='Short',closeLong='closeLong', closeShort='closeShort'\
                 , mhp=None,sl=None,sp=None):
    # step 1：实例化TimingDigger 通过output_folder和output_format指定测试报告的输出路径和输出格式，通过signal_name指定绩效文件名称

    #多空信号分别计算一遍 输出汇总结果
    td.process_signal(
        enter_signal=dv.get_ts(long),
        exit_signal=dv.get_ts(closeLong),
        sig_type="long",  # 信号类型 long/short
        price=dv.get_ts('close'),
        max_holding_period=mhp,  # 最大持有天数 可为空
        stoploss=-sl,  # 止损百分比 负数 可为空
        stopprofit=sp,  # 止盈百分比 正数 可为空
    )

    td.process_signal(
        enter_signal=dv.get_ts(short),
        exit_signal=dv.get_ts(closeShort),
        sig_type="short",  # 信号类型 long/short
        price=dv.get_ts("close"),
        max_holding_period=mhp,  # 最大持有天数 可为空
        stoploss=-sl,  # 止损百分比 负数 可为空
        stopprofit=sp,  # 止盈百分比 正数 可为空
    )


F:\Anaconda\envs\dayu\lib\site-packages\jaqs\research\signaldigger\plotting.py:10: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  mpl.use('Agg')
F:\Anaconda\envs\dayu\lib\site-packages\matplotlib\__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [3]:
start = time()
path = r'./min_data/VnTrader_1Min_Db'
props = {'fields': 'open,high,low,close,volume','symbol': 'BTCUSDT:binance', 'freq': '1D',
         'start_date':20180101000000}

Time_dict = ds.bar_reader(path,props)
print(time()-start)

8.618040323257446


In [4]:
from jaqs_fxdayu.data.hf_dataview import HFDataView
dv = HFDataView()

In [5]:
dv.create_init_dv(Time_dict.set_index(["trade_date","symbol"]))
data1D = Time_dict.set_index('datetime')

Initialize dataview success.


In [6]:
REVS5M20_J = dv.add_formula('REVS5M20_A','close/Delay(close,5)-close/Delay(close,20)', add_data=True)

进场信号：五日动量和20日动量的差值大于0，说明最近的上升的势头很猛
出场信号：五日动量和20日动量的差值小于于0，说明最近的下降的势头很猛

In [7]:
Long = dv.add_formula('Long','If(REVS5M20_A>0, 2, 0)', add_data=True)
Short = dv.add_formula('Short','If(REVS5M20_A<0, -2, 0)', add_data=True)

close_long = dv.add_formula("closeLong","If(Short==-2,1,0)", add_data=True)
close_short = dv.add_formula("closeShort","If(Long==2,-1,0)", add_data=True)

In [8]:
td = TimingDigger(output_folder=".", output_format='pdf',signal_name='REVS5M20_A')
TimingSignal(td, dv, long='Long', short='Short',closeLong='closeLong', closeShort='closeShort',mhp=100, sl=0.05, sp=0.3)#调取数据

Nan Data Count (should be zero) : 0;  Percentage of effective data: 53%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%


In [9]:
td.create_event_report(sig_type="long")

*****-Summary-*****
Event Analysis
                       win    loss      all
t-stat               8.443 -25.343    0.221
p-value              0.000   0.000    0.826
mean                 0.116  -0.074    0.002
std                  0.102   0.026    0.115
info_ratio           1.138  -2.782    0.019
skewness             1.080   0.217    1.389
kurtosis             0.065   0.244    1.381
pct5                 0.016  -0.119   -0.115
pct25                0.032  -0.089   -0.075
pct50                0.075  -0.073   -0.054
pct75                0.170  -0.058    0.066
pct95                0.330  -0.021    0.253
occurance           56.000  84.000  140.000
win_ratio              NaN     NaN    0.400
win_mean/loss_mean     NaN     NaN    1.573


In [10]:
td.create_event_report(sig_type="short")

*****-Summary-*****
Event Analysis
                       win    loss     all
t-stat              10.279 -14.868   2.336
p-value              0.000   0.000   0.022
mean                 0.104  -0.061   0.024
std                  0.069   0.027   0.098
info_ratio           1.499  -2.241   0.244
skewness             0.104   0.300   0.512
kurtosis            -1.256   0.216  -1.029
pct5                 0.004  -0.106  -0.096
pct25                0.042  -0.072  -0.060
pct50                0.097  -0.060   0.003
pct75                0.154  -0.053   0.102
pct95                0.210  -0.004   0.205
occurance           48.000  45.000  93.000
win_ratio              NaN     NaN   0.516
win_mean/loss_mean     NaN     NaN   1.694


In [11]:
td.create_event_report(sig_type="long_short") #同时计算多空头信号的绩效

*****-Summary-*****
Event Analysis
                        win     loss      all
t-stat               12.659  -28.617    1.516
p-value               0.000    0.000    0.131
mean                  0.110   -0.069    0.011
std                   0.089    0.027    0.109
info_ratio            1.247   -2.529    0.100
skewness              0.978    0.257    1.081
kurtosis              0.427    0.261    0.566
pct5                  0.012   -0.117   -0.107
pct25                 0.032   -0.088   -0.069
pct50                 0.087   -0.065   -0.030
pct75                 0.168   -0.057    0.074
pct95                 0.309   -0.016    0.213
occurance           104.000  129.000  233.000
win_ratio               NaN      NaN    0.446
win_mean/loss_mean      NaN      NaN    1.591


可以看出输出的绩效报告之中win_ratio>0.4,win_mean/loss_mean>1.5，整体来说效果不错

In [12]:
start = time()
path = r'./min_data/VnTrader_1Min_Db'
props = {'fields': 'open,high,low,close,volume','symbol': 'EOSUSDT:binance', 'freq': '1D',
         'start_date':20180101000000}

Time_dict = ds.bar_reader(path,props)
print(time()-start)

5.390886306762695


In [13]:
dv.create_init_dv(Time_dict.set_index(["trade_date","symbol"]))
data1D = Time_dict.set_index('datetime')

Initialize dataview success.


In [14]:
REVS5M20_J = dv.add_formula('REVS5M20_A','close/Delay(close,5)-close/Delay(close,20)', add_data=True)

In [15]:
Long = dv.add_formula('Long','If(REVS5M20_A>0, 2, 0)', add_data=True)
Short = dv.add_formula('Short','If(REVS5M20_A<0, -2, 0)', add_data=True)

close_long = dv.add_formula("closeLong","If(Short==-2,1,0)", add_data=True)
close_short = dv.add_formula("closeShort","If(Long==2,-1,0)", add_data=True)

In [16]:
td = TimingDigger(output_folder=".", output_format='pdf',signal_name='REVS5M20_A1')
TimingSignal(td, dv, long='Long', short='Short',closeLong='closeLong', closeShort='closeShort',mhp=100, sl=0.05, sp=0.3)#调取数据

Nan Data Count (should be zero) : 0;  Percentage of effective data: 57%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 21%


In [17]:
td.create_event_report(sig_type="long")

*****-Summary-*****
Event Analysis
                       win    loss     all
t-stat               6.368 -13.949  -0.037
p-value              0.000   0.000   0.971
mean                 0.217  -0.100  -0.001
std                  0.152   0.048   0.174
info_ratio           1.424  -2.079  -0.005
skewness            -0.283  -0.269   1.220
kurtosis            -1.665  -1.074   0.118
pct5                 0.007  -0.174  -0.171
pct25                0.035  -0.147  -0.120
pct50                0.303  -0.086  -0.064
pct75                0.364  -0.064   0.027
pct95                0.393  -0.028   0.374
occurance           21.000  46.000  67.000
win_ratio              NaN     NaN   0.313
win_mean/loss_mean     NaN     NaN   2.165


In [18]:
td.create_event_report(sig_type="short")

*****-Summary-*****
Event Analysis
                       win    loss     all
t-stat               3.878  -7.850   1.221
p-value              0.002   0.000   0.234
mean                 0.093  -0.067   0.027
std                  0.087   0.025   0.104
info_ratio           1.076  -2.617   0.255
skewness             0.832   1.351   0.846
kurtosis            -0.777   1.737  -0.219
pct5                 0.001  -0.097  -0.093
pct25                0.027  -0.072  -0.070
pct50                0.067  -0.071   0.009
pct75                0.134  -0.064   0.072
pct95                0.244  -0.024   0.239
occurance           14.000  10.000  24.000
win_ratio              NaN     NaN   0.583
win_mean/loss_mean     NaN     NaN   1.396


In [19]:
td.create_event_report(sig_type="long_short")

*****-Summary-*****
Event Analysis
                       win    loss     all
t-stat               6.802 -14.942   0.382
p-value              0.000   0.000   0.703
mean                 0.167  -0.094   0.006
std                  0.143   0.047   0.159
info_ratio           1.167  -2.015   0.040
skewness             0.312  -0.408   1.163
kurtosis            -1.544  -0.711   0.266
pct5                 0.005  -0.172  -0.168
pct25                0.031  -0.127  -0.095
pct50                0.095  -0.077  -0.059
pct75                0.314  -0.063   0.053
pct95                0.382  -0.023   0.369
occurance           35.000  56.000  91.000
win_ratio              NaN     NaN   0.385
win_mean/loss_mean     NaN     NaN   1.777


在新的品种中同样的进出场信号，可以发现long和long_short的绩效报告中win_ratio都略小于0.4，但是他们的win_mean/loss_mean都远比1大，即使牺牲一点胜率但是期望赚钱的数量是可以接受的，也就是在新品种中这个信号也适用